In [1]:
# fetching web pages
import requests
# getting/processing html from web pages
from bs4 import BeautifulSoup
# processing text with regular expressions
import re

In [2]:
# setup variables
# base url
root='https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/'
# how many stocks are there
numstocks=30
# target tags
targettags=['h3', 'p']
# success code
success=200

In [3]:
# check
for i in range(1, numstocks+1):
    url=root+str(i)
    print (url)

https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/1
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/3
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/4
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/5
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/6
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/7
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/8
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/9
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q

In [159]:
# prototype with first page
for i in range(22,23): #(1,numstocks+1)
    url=root+str(i)
    page=requests.get(url)
    if (page.status_code == success):
        print('Visited URL: {}'.format(page.url))
        html = BeautifulSoup(page.content, 'html.parser')
        # extract the <h3> tag
        h3=html.body.h3
        # extract the <p> tag
        if i==1: # hack for the first page
            p=html.select('p')[18]
        else: p=html.body.p
    else:
        print ('Oops, got error: {}', format(page.status_code))
print ("Done!")

Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/22/
Done!


In [160]:
print (h3,'\n')
# extract text from h3 tag
print (h3.text)

<h3 style="text-align: justify;">9.  PayPal Holdings, Inc. (NASDAQ: <a href="https://www.insidermonkey.com/insider-trading/company/paypal%20holdings%20inc/1633917/">PYPL</a>): $290.81</h3> 

9.  PayPal Holdings, Inc. (NASDAQ: PYPL): $290.81


In [164]:
def parseheader(h3):
    # print (h3)
    # convert to string
    h3text=h3.text
    # print (h3text)
    # grab digits at the front of the string (rank)
    rank=re.search('^\d*', h3text).group()
    # print ("rank:", rank )
    # grab the $ price - dollar, zero or more digits, decimal, zero or more digits
    price=re.search('\$\d*.*\d*', h3text).group().strip('\$')
    # print ("price:", price)
    # grab colon separated exchange followed by ticker
    exchangeticker=re.sub(' ','', re.search('\([A-Z]*: [A-Z]*\)', h3text).group())
    # print ("exchange and ticker:", exchangeticker)
    exchange, ticker=re.sub('\(|\)','', exchangeticker).split(':')
    # print ("ticker:", ticker)
    # print ("exchange:", exchange)
    # google finance compatible ticker
    gfinanceticker=ticker+":"+exchange
    # print ("Google Finance compatible ticker:", gfinanceticker)
    # grab the stock shortname (this is ugly, but regex will probably not work here)
    shortname=h3text.split('(')[0].split('.',1)[1].strip()
    # print (shortname)
    # print (rank, longticker, shortname, price)
    return (rank, gfinanceticker, shortname, price)

In [165]:
rank, gfinanceticker, shortname, price = parseheader(h3)
print (rank, gfinanceticker, shortname, price)

9 PYPL:NASDAQ PayPal Holdings, Inc. 290.81


In [9]:
print (p)

<p style="text-align: justify;">Number of Hedge Funds: 92 (2020Q4)<br/>
Number of Hedge Funds: 71 (2020Q3)<br/>
Total Dollar Amount of Long Hedge Fund Positions: $7.2 billion<br/>
Percent of Hedge Funds with Long Positions: 10.4%<br/>
First Quarter Return (through February 18): 12.9%<br/>
Popularity Ranking (2020Q3): 59<br/>
Popularity Ranking (2020Q2): 41<br/>
Noteworthy Hedge Fund Shareholders: <a href="https://www.insidermonkey.com/hedge-fund/tiger+global+management+llc/25/">Tiger Global Management LLC</a>, <a href="https://www.insidermonkey.com/hedge-fund/coatue+management/79/">Coatue Management</a></p>


In [ ]:
def parsehedgefunds(input):
    tokens=re.split(':|\(|\)', input.replace(u'\xa0', '').strip('\n'))
    numfundsname=(tokens[0].replace(" ","-")+'-'+tokens[2]).lower()
    numfundsvalue=re.sub(' ','', tokens[1])
    return(numfundsname, numfundsvalue)

In [ ]:
def parsepositions(input):
    # tokens=re.split(':|\(|\)', input.strip('\n'))
    tokens=re.split(':|\$|\%', input.replace(u'\xa0', '').strip('\n'))
    # print (tokens)
    positionsname=(tokens[0].replace(" ","-")+'-billions').lower()
    positionsvalue=tokens[2].split()[0]
    return(positionsname, positionsvalue)

In [ ]:
"""psplit1=pcontents[4].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
name=(psplit1[0].replace(" ","-")+'-billions').lower()
value=psplit2[1].strip('$')
print (name, value)
"""

In [ ]:
p=html.body.p
pcontents=p.contents
print (pcontents)

['Number of Hedge Funds:\xa0 94 (2020Q4)', <br/>, '\nNumber of Hedge Funds: 71 (2020Q3)', <br/>, '\nTotal Dollar Amount of Long Hedge Fund Positions: $5.0 billion', <br/>, '\nPercent of Hedge Funds with Long Positions: 10.6%', <br/>, '\nFirst Quarter Return (through February 18): 31%', <br/>, '\nPopularity Ranking (2020Q3): 60', <br/>, '\nPopularity Ranking (2020Q2): 68', <br/>, '\nNoteworthy Hedge Fund Shareholders: ', <a href="https://www.insidermonkey.com/hedge-fund/scge+management/555/">SCGE Management</a>, ', ', <a href="https://www.insidermonkey.com/hedge-fund/alkeon+capital+management/281/">Alkeon Capital Management</a>]


In [ ]:
from html.parser import HTMLParser

In [ ]:
HTMLParser.feed(p)

TypeError: feed() missing 1 required positional argument: 'data'

In [ ]:
print (pcontents[0])
print (re.match([\s:(]))

SyntaxError: unexpected character after line continuation character (<ipython-input-174-b8e20f0e7bfc>, line 2)

In [ ]:
print (pcontents[0])
name, value = parsehedgefunds(pcontents[0])
print (name, value)

print (pcontents[2])
name, value = parsehedgefunds(pcontents[2])
print (name, value)

print (pcontents[4])
name, value = parsepositions(pcontents[4])
print (name, value)

print (pcontents[6])
name, value = parsepositions(pcontents[6])
print (name, value)

Number of Hedge Funds:  94 (2020Q4)
number-of-hedge-funds-2020q4 94

Number of Hedge Funds: 71 (2020Q3)
number-of-hedge-funds-2020q3 71

Total Dollar Amount of Long Hedge Fund Positions: $5.0 billion
total-dollar-amount-of-long-hedge-fund-positions-billions 5.0

Percent of Hedge Funds with Long Positions: 10.6%


IndexError: list index out of range

In [ ]:
# get the *first* occurence of paragraph tag
p=html.body.p

In [ ]:
pcontents=p.contents

In [ ]:
type (pcontents)

list

In [ ]:
type (pcontents)

list

In [ ]:
import re

In [ ]:
print (pcontents[4])
psplit1=pcontents[4].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
name=(psplit1[0].replace(" ","-")+'-billions').lower()
value=psplit2[1].strip('$')
print (name, value)


Total Dollar Amount of Long Hedge Fund Positions: $51.5 billion
total-dollar-amount-of-long-hedge-fund-positions-billions 51.5


In [ ]:
print (pcontents[6])
psplit1=pcontents[6].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
name=(psplit1[0].replace(" ","-")).lower()
value=psplit2[1].strip('%')
print (name, value)


Percent of Hedge Funds with Long Positions: 30.8%
percent-of-hedge-funds-with-long-positions 30.8


In [ ]:
print (pcontents[8])
psplit1=pcontents[8].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
psplit3=psplit1[0].split('(')
#print (psplit3)
name=(psplit3[0]+psplit3[1].strip('\)')).replace(" ","-").lower()
value=psplit2[1].strip('%')
print (name, value)


First Quarter Return (through February 18): 2.2%
first-quarter-return-through-february-18 2.2


In [ ]:
print (pcontents[10])
psplit1=pcontents[10].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
psplit3=psplit1[0].split('(')
#print (psplit3)
name=(psplit3[0]+psplit3[1].strip('\)')).replace(" ","-").lower()
value=psplit2[1].strip('%')
print (name, value)


Popularity Ranking (2020Q3): 1
popularity-ranking-2020q3 1


In [ ]:
print (pcontents[12])
psplit1=pcontents[12].strip('\n').split(':')
#print (psplit1)
psplit2=psplit1[1].split(' ')
#print (psplit2)
psplit3=psplit1[0].split('(')
#print (psplit3)
name=(psplit3[0]+psplit3[1].strip('\)')).replace(" ","-").lower()
value=psplit2[1].strip('%')
print (name, value)


Popularity Ranking (2020Q2): 1
popularity-ranking-2020q2 1


In [ ]:
print (pcontents)

['Number of Hedge Funds: 273 (2020Q4)', <br/>, '\nNumber of Hedge Funds: 245 (2020Q3)', <br/>, '\nTotal Dollar Amount of Long Hedge Fund Positions: $51.5 billion', <br/>, '\nPercent of Hedge Funds with Long Positions: 30.8%', <br/>, '\nFirst Quarter Return (through February 18): 2.2%', <br/>, '\nPopularity Ranking (2020Q3): 1', <br/>, '\nPopularity Ranking (2020Q2): 1', <br/>, '\nNoteworthy Hedge Fund Shareholders: Masayoshi Son']


In [ ]:
print (pcontents[14])
psplit1=pcontents[14].strip('\n').split(':')
# print (psplit1)
name=psplit1[0].replace(" ","-").lower()
value=psplit1[1]
print (name, value)


Noteworthy Hedge Fund Shareholders: Masayoshi Son
noteworthy-hedge-fund-shareholders  Masayoshi Son


In [ ]:
print (targettags)

['h3', 'p']


In [ ]:
for target in targettags:
            stocks.append(html.select(target))

In [ ]:
stocks = []

for i in range(1,3): #(1,numstocks+1)
    url=root+str(i)
    page=requests.get(url)
    if (page.status_code == success):
        print('Visited URL: {}'.format(page.url))
        html = BeautifulSoup(page.content, 'html.parser')
        h3=html.body.h3
        if i==1: # hack for the first page
            p=html.select('p')[18]
        else: p=html.body.p
    else:
        print ('Oops, got error: {}', format(page.status_code))
    
    rank, longticker, shortname, price = parseheader(h3)
    print (rank, longticker, shortname, price)
    
    name, value = parseparagraph(p)
print ("Done!")

Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/
30 CRWD:NASDAQ CrowdStrike Holdings, Inc. 239.25
['Number of Hedge Funds: 92 (2020Q4)', <br/>, '\nNumber of Hedge Funds: 71 (2020Q3)', <br/>, '\nTotal Dollar Amount of Long Hedge Fund Positions: $7.2 billion', <br/>, '\nPercent of Hedge Funds with Long Positions: 10.4%', <br/>, '\nFirst Quarter Return (through February 18): 12.9%', <br/>, '\nPopularity Ranking (2020Q3): 59', <br/>, '\nPopularity Ranking (2020Q2): 41', <br/>, '\nNoteworthy Hedge Fund Shareholders: ', <a href="https://www.insidermonkey.com/hedge-fund/tiger+global+management+llc/25/">Tiger Global Management LLC</a>, ', ', <a href="https://www.insidermonkey.com/hedge-fund/coatue+management/79/">Coatue Management</a>]
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2/
29 TWLO:NYSE Twilio Inc. 443.49
['Number of Hedge Funds:\xa0 94 (2020Q4)', <br/>,

In [ ]:
print (stocks)

[[<h3 style="text-align: justify;">30. CrowdStrike Holdings, Inc. (NASDAQ: <a href="https://www.insidermonkey.com/insider-trading/company/crowdstrike%20holdings%20inc/1535527/">CRWD</a>): $239.25</h3>], [<p style="text-align: justify;">Hedge funds started off 2021 in a lot of flux, inviting the wrath of retail investors and Reddit’s meme-stock armies amid the GameStop saga, adding to their disrepute among average investors. But unlike amateur investors who usually make their opinions based on the news cycle, prudent analysts know that barring a few big names, hedge funds gained a lot in the GameStop fight with retail investors. But there’s more to the hedge funds’ performance than the GameStop factor.</p>, <p style="text-align: justify;">In 2020, global hedge funds gained around 12%, posting their strongest full-year returns since 2009. Close to 40% of the managers in the Eurekahedge Hedge Fund Index reported double-digit return for 2020. Data from LCH Investments show that the world’s

In [ ]:
type (stocks)

list

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(stocks)

TypeError: Expected list, got ResultSet